In [1]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [2]:
total_problem_number_list = [i for i in range(1000, 31285)]
total_problem_name_list = []
total_problem_tier_list = []
total_problem_algorithm_list = []
total_crawling_problem_number_list = []
total_problem_time_condition_list = []
total_problem_memory_condition_list = []
total_problem_submission_count_list = []
total_problem_answer_submission_count_list = []
total_problem_answered_people_count_list = []
total_problem_correct_rate_list = []

total_problem_description_list = []
total_problem_input_list = []
total_problem_output_list = []

remove_problem_list = []
total_english_problem_list = []

In [2]:
total_problem_number_list = [i for i in range(1000, 31285)]

with open('DB_problem_table.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

total_problem_name_list = loaded_data['total_problem_name_list']
total_problem_tier_list = loaded_data['total_problem_tier_list']
total_problem_algorithm_list = loaded_data['total_problem_algorithm_list']
total_crawling_problem_number_list = loaded_data['total_crawling_problem_number_list']
total_problem_time_condition_list = loaded_data['total_problem_time_condition_list']
total_problem_memory_condition_list = loaded_data['total_problem_memory_condition_list']
total_problem_submission_count_list = loaded_data['total_problem_submission_count_list']
total_problem_answer_submission_count_list = loaded_data['total_problem_answer_submission_count_list']
total_problem_answered_people_count_list = loaded_data['total_problem_answered_people_count_list']
total_problem_correct_rate_list = loaded_data['total_problem_correct_rate_list']
total_problem_description_list = loaded_data['total_problem_description_list']
total_problem_input_list = loaded_data['total_problem_input_list']
total_problem_output_list = loaded_data['total_problem_output_list']
remove_problem_list = loaded_data['remove_problem_list']
total_english_problem_list = loaded_data['total_english_problem_list']



In [3]:
tier_dict = {
    'Not ratable':0,
    'Bronze V': 1,
    'Bronze IV': 2,
    'Bronze III': 3,
    'Bronze II': 4,
    'Bronze I': 5,
    'Silver V': 6,
    'Silver IV': 7,
    'Silver III': 8,
    'Silver II': 9,
    'Silver I': 10,
    'Gold V': 11,
    'Gold IV': 12,
    'Gold III': 13,
    'Gold II': 14,
    'Gold I': 15,
    'Platinum V': 16,
    'Platinum IV': 17,
    'Platinum III': 18,
    'Platinum II': 19,
    'Platinum I': 20,
    'Diamond V': 21,
    'Diamond IV': 22,
    'Diamond III': 23,
    'Diamond II': 24,
    'Diamond I': 25,
    'Ruby V': 26,
    'Ruby IV': 27,
    'Ruby III': 28,
    'Ruby II': 29,
    'Ruby I': 30,
    'Master':31
}
flipped_tier_dict = {
    0: 'Not ratable',
    1: 'Bronze V',
    2: 'Bronze IV',
    3: 'Bronze III',
    4: 'Bronze II',
    5: 'Bronze I',
    6: 'Silver V',
    7: 'Silver IV',
    8: 'Silver III',
    9: 'Silver II',
    10: 'Silver I',
    11: 'Gold V',
    12: 'Gold IV',
    13: 'Gold III',
    14: 'Gold II',
    15: 'Gold I',
    16: 'Platinum V',
    17: 'Platinum IV',
    18: 'Platinum III',
    19: 'Platinum II',
    20: 'Platinum I',
    21: 'Diamond V',
    22: 'Diamond IV',
    23: 'Diamond III',
    24: 'Diamond II',
    25: 'Diamond I',
    26: 'Ruby V',
    27: 'Ruby IV',
    28: 'Ruby III',
    29: 'Ruby II',
    30: 'Ruby I',
    31: 'Master'
 }


In [13]:
driver = webdriver.Chrome('chromedriver.exe')
#driver = webdriver.Chrome('chromedriver.exe', options=chrome_options)
driver.get("https://www.acmicpc.net/login?next=%2Fproblem%2F1000")

username_input = driver.find_element_by_name("login_user_id")  # 사용자명 필드의 name 속성값에 맞게 수정
password_input = driver.find_element_by_name("login_password")  # 비밀번호 필드의 name 속성값에 맞게 수정
login_button = driver.find_element_by_xpath("//*[@id='submit_button']")  # 로그인 버튼의 XPath에 맞게 수정

username_input.send_keys("llsy159")
password_input.send_keys("llsy13579!")
login_button.click()

In [14]:

for n in total_problem_number_list:

    if((n in total_crawling_problem_number_list) or (n in total_english_problem_list) or (n in remove_problem_list)):
        continue

    problem_url = 'https://www.acmicpc.net/problem/' + str(n)
    print(problem_url)
   
    driver.get(problem_url)

    try:
        problem_description = driver.find_element_by_css_selector('#problem_description')
    except NoSuchElementException:
        remove_problem_list.append(n)
        print('존재하지 않는 문제')
        continue
    
    if(not any(char >= '가' and char <= '힣' for char in problem_description.text)):

        total_english_problem_list.append(n)
        print('영어 문제')
        continue

    problem_info_object = driver.find_element_by_css_selector('#problem-info')
    problem_info_list = [float(pio) if '.' in pio else int(pio) for pio in re.findall(r'\d+\.\d+|\d+', problem_info_object.text.split('\n')[1])]
    
    if(len(problem_info_list)!=6):
        
        remove_problem_list.append(n)
        print('정상적이지 않은 문제')
        continue
    
    problem_name_object = driver.find_element_by_css_selector('#problem_title')
    problem_tier_object = driver.find_element_by_css_selector('.solvedac-tier')


    
    total_problem_name_list.append(problem_name_object.text)
    try:
        total_problem_tier_list.append(flipped_tier_dict[int(re.search(r'/tier/(\d+)\.svg', problem_tier_object.get_attribute('src')).group(1))])
    except AttributeError:
        total_problem_tier_list.append('Zero')
    
    
    try:
        problem_algorithm_object = driver.find_element_by_css_selector('.show-spoiler')
        algorithm_button = driver.find_element_by_xpath("//*[@id='problem_tags']/div[2]/p/a")
        algorithm_button.click()
        problem_algorithm_object = driver.find_element_by_css_selector('.spoiler-list')
        total_problem_algorithm_list.append(problem_algorithm_object.text.split('\n'))
    except NoSuchElementException: 
        try:
            problem_algorithm_object = driver.find_element_by_css_selector('.spoiler-list')
            total_problem_algorithm_list.append(problem_algorithm_object.text.split('\n'))
        except NoSuchElementException: 
            total_problem_algorithm_list.append('Not Exist Algorithm')

    
    total_problem_time_condition_list.append(problem_info_list[0])
    total_problem_memory_condition_list.append(problem_info_list[1])
    total_problem_submission_count_list.append(problem_info_list[2])
    total_problem_answer_submission_count_list.append(problem_info_list[3])
    total_problem_answered_people_count_list.append(problem_info_list[4])
    total_problem_correct_rate_list.append(problem_info_list[5])
    total_crawling_problem_number_list.append(n)

    total_problem_description_list.append(problem_description.text)

    try:
        problem_input = driver.find_element_by_css_selector('#problem_input')
        total_problem_input_list.append(problem_input.text)
    except NoSuchElementException:
        total_problem_input_list.append("정해진 입력 형식이 없습니다.")

    try:
        problem_output = driver.find_element_by_css_selector('#problem_output')
        total_problem_output_list.append(problem_output.text)
    except NoSuchElementException:
        total_problem_output_list.append("정해진 출력 형식이 없습니다.")   

    
    lists_to_save = {
        'total_problem_name_list': total_problem_name_list,
        'total_problem_tier_list': total_problem_tier_list,
        'total_problem_algorithm_list': total_problem_algorithm_list,
        'total_crawling_problem_number_list': total_crawling_problem_number_list,
        'total_problem_time_condition_list': total_problem_time_condition_list,
        'total_problem_memory_condition_list': total_problem_memory_condition_list,
        'total_problem_submission_count_list': total_problem_submission_count_list,
        'total_problem_answer_submission_count_list': total_problem_answer_submission_count_list,
        'total_problem_answered_people_count_list': total_problem_answered_people_count_list,
        'total_problem_correct_rate_list': total_problem_correct_rate_list,
        'total_problem_description_list' : total_problem_description_list,
        'total_problem_input_list' : total_problem_input_list,
        'total_problem_output_list' : total_problem_output_list,
        'remove_problem_list': remove_problem_list,
        'total_english_problem_list': total_english_problem_list
    }

    file_name = "DB_problem_table.pkl"
    with open(file_name, 'wb') as file:
        pickle.dump(lists_to_save, file) 
    

#driver.quit()

https://www.acmicpc.net/problem/30831
https://www.acmicpc.net/problem/30832
https://www.acmicpc.net/problem/30833
영어 문제
https://www.acmicpc.net/problem/30834
영어 문제
https://www.acmicpc.net/problem/30835
영어 문제
https://www.acmicpc.net/problem/30836
영어 문제
https://www.acmicpc.net/problem/30837
영어 문제
https://www.acmicpc.net/problem/30838
영어 문제
https://www.acmicpc.net/problem/30839
영어 문제
https://www.acmicpc.net/problem/30840
영어 문제
https://www.acmicpc.net/problem/30841
영어 문제
https://www.acmicpc.net/problem/30842
영어 문제
https://www.acmicpc.net/problem/30843
영어 문제
https://www.acmicpc.net/problem/30844
영어 문제
https://www.acmicpc.net/problem/30845
영어 문제
https://www.acmicpc.net/problem/30846
영어 문제
https://www.acmicpc.net/problem/30847
영어 문제
https://www.acmicpc.net/problem/30848
영어 문제
https://www.acmicpc.net/problem/30849
영어 문제
https://www.acmicpc.net/problem/30850
영어 문제
https://www.acmicpc.net/problem/30851
영어 문제
https://www.acmicpc.net/problem/30852
영어 문제
https://www.acmicpc.net/problem/30853
영어 문제


In [ ]:
total_problem_name_list.pop(-1)

'정ㅋ벅ㅋ'

In [ ]:
total_problem_tier_list.pop(-1)

In [23]:
c_n=-1
print(total_problem_name_list[c_n])
print(total_problem_tier_list[c_n])
print(total_problem_algorithm_list[c_n])
print(total_crawling_problem_number_list[c_n])
print(total_problem_time_condition_list[c_n])
print(total_problem_memory_condition_list[c_n])
print(total_problem_submission_count_list[c_n])
print(total_problem_answer_submission_count_list[c_n])
print(total_problem_answered_people_count_list[c_n])
print(total_problem_correct_rate_list[c_n])
print(total_problem_description_list[c_n])
print(total_problem_input_list[c_n])
print(total_problem_output_list[c_n])
print(remove_problem_list[c_n])
print(total_english_problem_list[c_n])

print(len(total_problem_name_list))
print(len(total_problem_tier_list))
print(len(total_problem_algorithm_list))
print(len(total_crawling_problem_number_list))
print(len(total_problem_time_condition_list))
print(len(total_problem_memory_condition_list))
print(len(total_problem_submission_count_list))
print(len(total_problem_answer_submission_count_list))
print(len(total_problem_answered_people_count_list))
print(len(total_problem_correct_rate_list))
print(len(total_problem_description_list))
print(len(total_problem_input_list))
print(len(total_problem_output_list))
print(len(remove_problem_list))
print(len(total_english_problem_list))

과일 게임
Diamond II
['구현', '자료 구조', '세그먼트 트리']
31284
4
1024
4
4
3
100.0
과일 게임은 여러 가지 종류의 과일들을 합쳐 크기가 큰 종류의 과일을 만드는 게임이다. 과일 게임의 게임판은 수열
X
[
0
]
,
X
[
1
]
,
⋯
,
X
[
K
−
1
]
$X[0], X[1], \cdots , X[K - 1]$로 표현할 수 있다. 이 때 각 수는 과일의 종류에 따른 번호를 나타내며, 번호가 클수록 과일의 크기가 크다는 것을 의미한다.
이 때 플레이어는 종류가 같으며 인접한 두 과일을 합쳐서 크기가 큰 과일을 만들 수 있는 합치기 연산을 수행할 수 있다. 이 연산은 다음과 같이 정의된다.
합치기:
X
[
0
]
,
X
[
1
]
,
⋯
,
X
[
K
−
1
]
$X[0], X[1], \cdots , X[K - 1]$로 표현되는 게임판에서 정수
0
≤
i
≤
K
−
2
$0 ≤ i ≤ K - 2$를 골라,
X
[
i
]
=
X
[
i
+
1
]
$X[i] = X[i + 1]$을 만족한다면 게임판을
X
[
0
]
,
⋯
,
X
[
i
−
1
]
,
X
[
i
]
+
1
,
X
[
i
+
2
]
,
⋯
,
X
[
K
−
1
]
$X[0], \cdots , X[i - 1], X[i] + 1, X[i + 2], \cdots , X[K - 1]$로 바꾼다.
플레이어의 목표는 초기 게임판이 주어지면 합치기 연산을
0
$0$회 이상 사용하여 크기가 큰 과일을 만드는 것이다.
예를 들어서 게임판이
X
=
[
2
,
1
,
1
,
3
,
2
]
$X = [2, 1, 1, 3, 2]$인 경우,
X
[
1
]
=
X
[
2
]
$X[1] = X[2]$이기 때문에
i
=
1
$i = 1$을 선택하여 합치기 연산을 수행하면 게임판이
X
=
[
2
,
2
,
3
,
2
]
$X = [2, 2, 3, 2]$로 바뀌게 된다. 또
X
[
0
]
=
X
[
1
]
$X[0] = X[1]$이기 때문에
i
=
0
$i = 0$을 선택하여 합치기 연산을 

In [25]:
total_problem_input_list[-1]

''

In [34]:
data = {
        'problem_number': total_crawling_problem_number_list,
        'problem_name': total_problem_name_list,
        'problem_tier': total_problem_tier_list,
        'problem_algorithm': total_problem_algorithm_list,
        'problem_time_condition': total_problem_time_condition_list,
        'problem_memory_condition': total_problem_memory_condition_list,
        'problem_submission_count': total_problem_submission_count_list,
        'problem_answer_submission_count': total_problem_answer_submission_count_list,
        'problem_answered_people_count': total_problem_answered_people_count_list,
        'problem_correct_rate': total_problem_correct_rate_list,
        'problem_description' : total_problem_description_list,
        'problem_input' : total_problem_input_list,
        'problem_output' : total_problem_output_list,
    }

df = pd.DataFrame(data)
df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_description,problem_input,problem_output
0,1000,A+B,Bronze V,"[수학, 구현, 사칙연산]",2.00,128,1011580,401015,277523,39.424,"두 정수 A와 B를 입력받은 다음, A+B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A+B를 출력한다.
1,1001,A-B,Bronze V,"[수학, 구현, 사칙연산]",2.00,128,411847,282081,237360,69.768,"두 정수 A와 B를 입력받은 다음, A-B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A-B를 출력한다.
2,1002,터렛,Silver III,"[수학, 기하학, 많은 조건 분기]",2.00,128,208940,46586,35834,22.467,조규현과 백승환은 터렛에 근무하는 직원이다. 하지만 워낙 존재감이 없어서 인구수는 ...,첫째 줄에 테스트 케이스의 개수\nT\n$T$가 주어진다. 각 테스트 케이스는 다음...,각 테스트 케이스마다 류재명이 있을 수 있는 위치의 수를 출력한다. 만약 류재명이 ...
3,1003,피보나치 함수,Silver III,[다이나믹 프로그래밍],0.25,128,210034,64283,50715,32.982,다음 소스는 N번째 피보나치 수를 구하는 C++ 함수이다.\nint fibonacc...,첫째 줄에 테스트 케이스의 개수 T가 주어진다.\n각 테스트 케이스는 한 줄로 이루...,각 테스트 케이스마다 0이 출력되는 횟수와 1이 출력되는 횟수를 공백으로 구분해서 ...
4,1004,어린 왕자,Silver III,"[수학, 기하학]",2.00,128,39519,17768,14955,46.005,어린 왕자는 소혹성 B-664에서 자신이 사랑하는 한 송이 장미를 위해 살아간다. ...,입력의 첫 줄에는 테스트 케이스의 개수 T가 주어진다. 그 다음 줄부터 각각의 테스...,각 테스트 케이스에 대해 어린 왕자가 거쳐야 할 최소의 행성계 진입/이탈 횟수를 출...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9493,31266,축구 대회,Platinum V,"[다이나믹 프로그래밍, 그리디 알고리즘, 브루트포스 알고리즘, 정렬, 비트마스킹, ...",1.00,1024,99,30,28,43.750,"설날을 맞아, 도훈이네 부대에서 축구 대회를 개최한다고 한다! 도훈이는 팀의 감독으...",첫 번째 줄에 부대 내 병사 수\nN\n$N$이 주어진다.\n이후\nN\n$N$개의...,팀 선발 조건에 맞게 선수들을 선발했을 때 팀의 만족도의 최댓값을 출력한다.
9494,31267,시간 외 근무 멈춰!!!,Platinum III,"[그리디 알고리즘, 이분 탐색, 매개 변수 탐색]",1.00,1024,74,21,12,27.273,"전역 전까지 시간 외 근무 지옥에서 벗어나지 못한 준민이는, 결국 전역을 앞두고 남...",첫 번째 줄에 후임병이 해야 하는 작업 개수\nN\n$N$과 데드라인 조작 횟수\n...,K\n$K$번 이하로 데드라인을 조작한 경우 민욱이가 해야 하는 최소 시간 외 근무...
9495,31268,물자 조달,Platinum III,"[그래프 이론, 데이크스트라, 최단 경로, 오프라인 쿼리, 플로이드–워셜]",1.00,1024,74,14,13,30.952,현재 A국과 B국 두 나라는 서로 전쟁 중이다. A국의 운전병 해찬이는 조국의 승리...,"첫 번째 줄에 부대의 수\nN\n$N$, 도로의 수\nM\n$M$, 쿼리의 수\nQ...",주어진 2번 쿼리의 출력값을 한 줄에 하나씩 출력한다.
9496,31283,평균 최대화,Ruby V,"[자료 구조, 기하학, 트리, 세그먼트 트리, 이분 탐색, 우선순위 큐, 작은 집합...",5.00,1024,2,2,2,100.000,양의 정수로 구성된 길이가\nm\n$m$ (\nm\n≥\n2\n$m ≥ 2$)인 수...,,


In [39]:
df[df['problem_input']=='정해진 입력 형식이 없습니다.']

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_description,problem_input,problem_output
5116,15596,정수 N개의 합,Bronze II,"[수학, 구현, 사칙연산]",0.1,512,141563,70339,60687,50.456,"정수 n개가 주어졌을 때, n개의 합을 구하는 함수를 작성하시오.\n작성해야 하는 ...",정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.
5174,15687,직사각형,Silver V,"[구현, 기하학]",1.0,512,2179,681,535,32.017,"직사각형의 정보를 저장할 클래스 Rectangle을 구현하시오.\nC++14, C+...",정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.
5524,16455,K번째 수 찾는 함수,Gold II,[무작위화],0.2,512,3539,368,118,11.569,"크기가 N인 수열 A1, A2, ..., AN이 주어진다. A를 오름차순 정렬했을 ...",정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.
6506,19618,벽 칠하기,Platinum I,"[다이나믹 프로그래밍, 그리디 알고리즘]",1.5,512,200,51,44,24.309,성렬이는 자기가 사는 집의 벽을 칠한지 한참 되었기 때문에 다시 칠하려고 한다. 벽...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.
6934,21863,__builtout_popcount,Platinum II,[비트마스킹],1.0,512,449,72,26,18.440,"GCC 내장 함수 중, __builtin_popcount(unsigned int x...",정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.
6945,21874,모자 게임,Gold II,"[수학, 애드 혹]",1.5,128,225,113,64,47.761,CS-House에서는 매주 목요일에 연세대학교 컴퓨터과학과에 대한 여러 이야기를 팟...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.
7619,24732,잠자는 규리,Platinum IV,"[자료 구조, 누적 합, 스택]",0.5,1024,187,27,22,22.680,2019년 송도의 세미나실에서는 오늘도 연세대학교 컴퓨터과학과 학생들이 모여 과제를...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.


In [29]:
df.info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9498 entries, 0 to 9497
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   problem_number                   9498 non-null   int64  
 1   problem_name                     9498 non-null   object 
 2   problem_tier                     9498 non-null   object 
 3   problem_algorithm                9498 non-null   object 
 4   problem_time_condition           9498 non-null   float64
 5   problem_memory_condition         9498 non-null   int64  
 6   problem_submission_count         9498 non-null   int64  
 7   problem_answer_submission_count  9498 non-null   int64  
 8   problem_answered_people_count    9498 non-null   int64  
 9   problem_correct_rate             9498 non-null   float64
 10  problem_description              9498 non-null   object 
 11  problem_input                    9498 non-null   object 
 12  problem_output      

In [40]:
df['problem_input'].replace('','정해진 입력 형식이 없습니다.', inplace=True)
df['problem_output'].replace('','정해진 출력 형식이 없습니다.', inplace=True)

In [43]:
df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_description,problem_input,problem_output
0,1000,A+B,Bronze V,"[수학, 구현, 사칙연산]",2.00,128,1011580,401015,277523,39.424,"두 정수 A와 B를 입력받은 다음, A+B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A+B를 출력한다.
1,1001,A-B,Bronze V,"[수학, 구현, 사칙연산]",2.00,128,411847,282081,237360,69.768,"두 정수 A와 B를 입력받은 다음, A-B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A-B를 출력한다.
2,1002,터렛,Silver III,"[수학, 기하학, 많은 조건 분기]",2.00,128,208940,46586,35834,22.467,조규현과 백승환은 터렛에 근무하는 직원이다. 하지만 워낙 존재감이 없어서 인구수는 ...,첫째 줄에 테스트 케이스의 개수\nT\n$T$가 주어진다. 각 테스트 케이스는 다음...,각 테스트 케이스마다 류재명이 있을 수 있는 위치의 수를 출력한다. 만약 류재명이 ...
3,1003,피보나치 함수,Silver III,[다이나믹 프로그래밍],0.25,128,210034,64283,50715,32.982,다음 소스는 N번째 피보나치 수를 구하는 C++ 함수이다.\nint fibonacc...,첫째 줄에 테스트 케이스의 개수 T가 주어진다.\n각 테스트 케이스는 한 줄로 이루...,각 테스트 케이스마다 0이 출력되는 횟수와 1이 출력되는 횟수를 공백으로 구분해서 ...
4,1004,어린 왕자,Silver III,"[수학, 기하학]",2.00,128,39519,17768,14955,46.005,어린 왕자는 소혹성 B-664에서 자신이 사랑하는 한 송이 장미를 위해 살아간다. ...,입력의 첫 줄에는 테스트 케이스의 개수 T가 주어진다. 그 다음 줄부터 각각의 테스...,각 테스트 케이스에 대해 어린 왕자가 거쳐야 할 최소의 행성계 진입/이탈 횟수를 출...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9493,31266,축구 대회,Platinum V,"[다이나믹 프로그래밍, 그리디 알고리즘, 브루트포스 알고리즘, 정렬, 비트마스킹, ...",1.00,1024,99,30,28,43.750,"설날을 맞아, 도훈이네 부대에서 축구 대회를 개최한다고 한다! 도훈이는 팀의 감독으...",첫 번째 줄에 부대 내 병사 수\nN\n$N$이 주어진다.\n이후\nN\n$N$개의...,팀 선발 조건에 맞게 선수들을 선발했을 때 팀의 만족도의 최댓값을 출력한다.
9494,31267,시간 외 근무 멈춰!!!,Platinum III,"[그리디 알고리즘, 이분 탐색, 매개 변수 탐색]",1.00,1024,74,21,12,27.273,"전역 전까지 시간 외 근무 지옥에서 벗어나지 못한 준민이는, 결국 전역을 앞두고 남...",첫 번째 줄에 후임병이 해야 하는 작업 개수\nN\n$N$과 데드라인 조작 횟수\n...,K\n$K$번 이하로 데드라인을 조작한 경우 민욱이가 해야 하는 최소 시간 외 근무...
9495,31268,물자 조달,Platinum III,"[그래프 이론, 데이크스트라, 최단 경로, 오프라인 쿼리, 플로이드–워셜]",1.00,1024,74,14,13,30.952,현재 A국과 B국 두 나라는 서로 전쟁 중이다. A국의 운전병 해찬이는 조국의 승리...,"첫 번째 줄에 부대의 수\nN\n$N$, 도로의 수\nM\n$M$, 쿼리의 수\nQ...",주어진 2번 쿼리의 출력값을 한 줄에 하나씩 출력한다.
9496,31283,평균 최대화,Ruby V,"[자료 구조, 기하학, 트리, 세그먼트 트리, 이분 탐색, 우선순위 큐, 작은 집합...",5.00,1024,2,2,2,100.000,양의 정수로 구성된 길이가\nm\n$m$ (\nm\n≥\n2\n$m ≥ 2$)인 수...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.


In [59]:
df = df.reset_index(drop=True)

In [60]:
df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_description,problem_input,problem_output
0,1000,A+B,Bronze V,"[수학, 구현, 사칙연산]",2.00,128,1011580,401015,277523,39.424,"두 정수 A와 B를 입력받은 다음, A+B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A+B를 출력한다.
1,1001,A-B,Bronze V,"[수학, 구현, 사칙연산]",2.00,128,411847,282081,237360,69.768,"두 정수 A와 B를 입력받은 다음, A-B를 출력하는 프로그램을 작성하시오.","첫째 줄에 A와 B가 주어진다. (0 < A, B < 10)",첫째 줄에 A-B를 출력한다.
2,1002,터렛,Silver III,"[수학, 기하학, 많은 조건 분기]",2.00,128,208940,46586,35834,22.467,조규현과 백승환은 터렛에 근무하는 직원이다. 하지만 워낙 존재감이 없어서 인구수는 ...,첫째 줄에 테스트 케이스의 개수\nT\n$T$가 주어진다. 각 테스트 케이스는 다음...,각 테스트 케이스마다 류재명이 있을 수 있는 위치의 수를 출력한다. 만약 류재명이 ...
3,1003,피보나치 함수,Silver III,[다이나믹 프로그래밍],0.25,128,210034,64283,50715,32.982,다음 소스는 N번째 피보나치 수를 구하는 C++ 함수이다.\nint fibonacc...,첫째 줄에 테스트 케이스의 개수 T가 주어진다.\n각 테스트 케이스는 한 줄로 이루...,각 테스트 케이스마다 0이 출력되는 횟수와 1이 출력되는 횟수를 공백으로 구분해서 ...
4,1004,어린 왕자,Silver III,"[수학, 기하학]",2.00,128,39519,17768,14955,46.005,어린 왕자는 소혹성 B-664에서 자신이 사랑하는 한 송이 장미를 위해 살아간다. ...,입력의 첫 줄에는 테스트 케이스의 개수 T가 주어진다. 그 다음 줄부터 각각의 테스...,각 테스트 케이스에 대해 어린 왕자가 거쳐야 할 최소의 행성계 진입/이탈 횟수를 출...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,31266,축구 대회,Platinum V,"[다이나믹 프로그래밍, 그리디 알고리즘, 브루트포스 알고리즘, 정렬, 비트마스킹, ...",1.00,1024,99,30,28,43.750,"설날을 맞아, 도훈이네 부대에서 축구 대회를 개최한다고 한다! 도훈이는 팀의 감독으...",첫 번째 줄에 부대 내 병사 수\nN\n$N$이 주어진다.\n이후\nN\n$N$개의...,팀 선발 조건에 맞게 선수들을 선발했을 때 팀의 만족도의 최댓값을 출력한다.
9492,31267,시간 외 근무 멈춰!!!,Platinum III,"[그리디 알고리즘, 이분 탐색, 매개 변수 탐색]",1.00,1024,74,21,12,27.273,"전역 전까지 시간 외 근무 지옥에서 벗어나지 못한 준민이는, 결국 전역을 앞두고 남...",첫 번째 줄에 후임병이 해야 하는 작업 개수\nN\n$N$과 데드라인 조작 횟수\n...,K\n$K$번 이하로 데드라인을 조작한 경우 민욱이가 해야 하는 최소 시간 외 근무...
9493,31268,물자 조달,Platinum III,"[그래프 이론, 데이크스트라, 최단 경로, 오프라인 쿼리, 플로이드–워셜]",1.00,1024,74,14,13,30.952,현재 A국과 B국 두 나라는 서로 전쟁 중이다. A국의 운전병 해찬이는 조국의 승리...,"첫 번째 줄에 부대의 수\nN\n$N$, 도로의 수\nM\n$M$, 쿼리의 수\nQ...",주어진 2번 쿼리의 출력값을 한 줄에 하나씩 출력한다.
9494,31283,평균 최대화,Ruby V,"[자료 구조, 기하학, 트리, 세그먼트 트리, 이분 탐색, 우선순위 큐, 작은 집합...",5.00,1024,2,2,2,100.000,양의 정수로 구성된 길이가\nm\n$m$ (\nm\n≥\n2\n$m ≥ 2$)인 수...,정해진 입력 형식이 없습니다.,정해진 출력 형식이 없습니다.


In [61]:
df.to_csv('../DataSet/final_DB_problem_table.csv',index=False)